# GeoPandas interface

Due to the nature of operations done with the vector data cubes, you will often find a need to convert Xarray object to GeoPandas GeoDataFrames. Xvec offers a set of methods extending support for exporting Xarray to pandas.

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr
import xvec

from geodatasets import get_path

Create a dummy DataArray with two dimensions indexed by geometry, mimicking transport counts between communities of Chicago.

In [2]:
chicago = gpd.read_file(get_path("geoda.chicago health"))

origin = destination = chicago.geometry.array
mode = ["car", "bike", "foot"]
date = pd.date_range("2023-01-01", periods=100)
hours = range(24)
rng = np.random.default_rng(1)
data = rng.integers(1, 100, size=(3, 100, 24, len(chicago), len(chicago)))
traffic_counts = xr.DataArray(
    data,
    coords=(mode, date, hours, origin, destination),
    dims=["mode", "date", "time", "origin", "destination"],
    name="traffic_counts",
).xvec.set_geom_indexes(["origin", "destination"], crs=chicago.crs)
traffic_counts

<xarray.DataArray 'traffic_counts' (mode: 3, date: 100, time: 24, origin: 77,
                                    destination: 77)>
array([[[[[47, 51, 75, ..., 72, 53, 87],
          [46, 37,  7, ...,  3, 49, 67],
          [45, 91, 95, ..., 18, 70, 77],
          ...,
          [39, 93, 18, ..., 85,  5, 76],
          [91, 96, 10, ..., 50, 96, 22],
          [66, 11,  6, ..., 35, 34, 15]],

         [[ 2, 30, 42, ..., 87, 37, 30],
          [ 2, 74, 64, ..., 78, 40,  3],
          [93, 86, 92, ..., 24, 63, 84],
          ...,
          [18, 32, 35, ..., 20, 65, 72],
          [ 9, 92, 95, ..., 63, 60, 59],
          [74, 82, 71, ..., 58, 27, 68]],

         [[45, 63, 94, ..., 61, 78, 60],
          [90, 50, 94, ..., 57, 97, 37],
          [47, 67, 73, ..., 35, 89, 70],
          ...,
...
          ...,
          [69, 11, 20, ..., 44, 86,  3],
          [75, 80, 55, ..., 50, 64, 58],
          [70, 12, 36, ..., 82, 25, 18]],

         [[40, 64, 76, ..., 90, 79, 88],
          [31, 96, 25, ..., 38, 50,  2],
          [36, 93, 99, ..., 31, 31, 38],
          ...,
          [43, 94, 80, ..., 39, 53, 34],
          [47, 13, 90, ..., 92, 94,  5],
          [27, 21, 59, ..., 46, 90, 31]],

         [[99, 42, 31, ..., 51, 74, 31],
          [46, 83, 96, ..., 11, 86, 20],
          [18, 38,  6, ..., 91, 48, 54],
          ...,
          [28, 88, 56, ...,  2,  4, 44],
          [88, 73, 55, ..., 17, 52,  9],
          [ 2, 29, 57, ..., 99, 50, 82]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * date         (date) datetime64[ns] 2023-01-01 2023-01-02 ... 2023-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object POLYGON ((-87.60914087617012 41.844692503461...
  * destination  (destination) object POLYGON ((-87.60914087617012 41.8446925...
Indexes:
    origin       GeometryIndex (crs=GEOGCS["WGS84(DD)",DATUM["WGS_1984",SPHEROID["WGS84",6378137,29 ...)
    destination  GeometryIndex (crs=GEOGCS["WGS84(DD)",DATUM["WGS_1984",SPHEROID["WGS84",6378137,29 ...)

Xvec offers `.xvec.to_geopandas()` and `.xvec.to_geodataframe()` methods as counterparts to the default `to_pandas()` and `to_dataframe()` methods built-in Xarray. While the former works on arrays with 2 dimensions or less, the latter is generalisable and always returns all coordinates. Consider following examples.

Select one day, one time and one mode and convert the data to a GeoDataFrame.

In [3]:
traffic_counts.sel(date="2023-02-28", time=12, mode="bike").xvec.to_geodataframe(
    geometry="origin"
)

,origin,destination,mode,date,time,traffic_counts
0,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...","POLYGON ((-87.60914 41.84469, -87.60915 41.844...",bike,2023-02-28,12,19
1,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...","POLYGON ((-87.59215 41.81693, -87.59231 41.816...",bike,2023-02-28,12,23
2,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...","POLYGON ((-87.62880 41.80189, -87.62879 41.801...",bike,2023-02-28,12,20
3,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...","POLYGON ((-87.60671 41.81681, -87.60670 41.816...",bike,2023-02-28,12,13
4,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...","POLYGON ((-87.59215 41.81693, -87.59215 41.816...",bike,2023-02-28,12,13
...,...,...,...,...,...,...
5924,"POLYGON ((-87.80676 42.00084, -87.80676 42.000...","POLYGON ((-87.69646 41.70714, -87.69644 41.706...",bike,2023-02-28,12,53
5925,"POLYGON ((-87.80676 42.00084, -87.80676 42.000...","POLYGON ((-87.64215 41.68508, -87.64249 41.685...",bike,2023-02-28,12,47
5926,"POLYGON ((-87.80676 42.00084, -87.80676 42.000...","MULTIPOLYGON (((-87.83658 41.98640, -87.83658 ...",bike,2023-02-28,12,26
5927,"POLYGON ((-87.80676 42.00084, -87.80676 42.000...","POLYGON ((-87.65456 41.99817, -87.65456 41.998...",bike,2023-02-28,12,75


You end up with two geometry column and because none is primary by default the cell above contains `geometry="origin"` to specify which of them shall become the active geometry of the GeoDataFrame. You can see that all coordinates are present, but `mode`, `date` and `time` are constants.

However, exporting the same using `.xvec.to_geopandas()` does not work because both index and columns are arrays of geometries.

In [4]:
traffic_counts.sel(date="2023-02-28", time=12, mode="bike").xvec.to_geopandas()

ValueError: Multiple coordinates based on xvec.GeometryIndex are not supported as GeoPandas.GeoDataFrame cannot be indexed by geometry. Try using `.xvec.to_geodataframe()` instead.

If you ensure that only one array of geometries is returned, you can use the method.

In [5]:
traffic_counts.sel(date="2023-02-28", time=12, origin=origin[0]).xvec.to_geopandas()

mode,destination,car,bike,foot
0,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",48,19,97
1,"POLYGON ((-87.59215 41.81693, -87.59231 41.816...",77,23,25
2,"POLYGON ((-87.62880 41.80189, -87.62879 41.801...",43,20,59
3,"POLYGON ((-87.60671 41.81681, -87.60670 41.816...",96,13,42
4,"POLYGON ((-87.59215 41.81693, -87.59215 41.816...",38,13,48
...,...,...,...,...
72,"POLYGON ((-87.69646 41.70714, -87.69644 41.706...",54,13,22
73,"POLYGON ((-87.64215 41.68508, -87.64249 41.685...",76,10,82
74,"MULTIPOLYGON (((-87.83658 41.98640, -87.83658 ...",49,82,14
75,"POLYGON ((-87.65456 41.99817, -87.65456 41.998...",84,28,16


Compared to the `to_geodataframe`, you can see that contstants are removed. and the final GeoDataFrame is much smaller.

In [6]:
traffic_counts.sel(date="2023-02-28", time=12, origin=origin[0]).xvec.to_geodataframe()

,destination,date,time,origin,traffic_counts
mode,,,,,
car,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",2023-02-28,12,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",48
car,"POLYGON ((-87.59215 41.81693, -87.59231 41.816...",2023-02-28,12,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",77
car,"POLYGON ((-87.62880 41.80189, -87.62879 41.801...",2023-02-28,12,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",43
car,"POLYGON ((-87.60671 41.81681, -87.60670 41.816...",2023-02-28,12,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",96
car,"POLYGON ((-87.59215 41.81693, -87.59215 41.816...",2023-02-28,12,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",38
...,...,...,...,...,...
foot,"POLYGON ((-87.69646 41.70714, -87.69644 41.706...",2023-02-28,12,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",22
foot,"POLYGON ((-87.64215 41.68508, -87.64249 41.685...",2023-02-28,12,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",82
foot,"MULTIPOLYGON (((-87.83658 41.98640, -87.83658 ...",2023-02-28,12,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",14


## Differences compares to the default pandas exports

While both methods wrap native Xarray methods `to_pandas` and `to_dataframe`, there are differences in behavior. Since GeoPandas doesn't support GeometryArray as an index (or columns), Xvec transforms the resulting DataFrame if that situation happens. It means either resetting the index to get geometries as a standard GeoSeries or transposing the DataFrame and then resetting the index (if geometries are indexing columns).